## Predicting Sale price using Linear and Random Forest regression

The Dataset Contains 5031 observations with 109 features.

In [883]:
# installing required packages
!pip install scikit-learn

# Import a bun of libraries.

import pandas as pd
import time
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_mldata
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
#from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

### Data Extraction

In [884]:
# Importing the training data
path = '/Users/mummadi/Downloads/Sammamish-Training.csv'
train = pd.read_csv(path)
train.head()

,ML Number,Status,List Price,Sold Price,Year,List Date,Sold Date,DOM,Address,City Name,...,Selling Office Name,Selling Office Phone,Senior Exemption,Sewer Company,Sewer Type,Show Addressto Public,Show Map Link,Site Features,Community,Sale Type
0,466258,Sold,4499000.0,4036000.0,2013,4/1/13,8/21/13,117.0,222 E 208 Lake Sammamish Pkwy SE,Sammamish,...,Windermere Real Estate/East,(425) 455-5300,False,City of Sammamish,Sewer Connected,True,True,"Cable TV, Deck, Disabled Access, Dog Run, Fenc...",East Lake Sammamish,MLS
1,435709,Sold,3700000.0,3685000.0,2013,1/8/13,1/11/13,NaN,333 E Lake Sammamish Pkwy SE,Sammamish,...,Windermere Real Estate/East,(425) 643-5500 x246,True,Sammamish Plateau,Available,True,True,"Cable TV, Dock, Gas Available, High Speed Inte...",Lake Sammamish,MLS
2,514713,Sold,2895000.0,2772500.0,2013,7/11/13,10/8/13,25.0,3839 E Lake Sammamish Shore Lane SE,Sammamish,...,Coldwell Banker Bain,(253) 851-9988,False,Samm Plateau Water & Sewer,Sewer Connected,True,True,"Boat House, Deck, Dock, Fenced-Partially, Gas ...",East Lake Sammamish,MLS
3,448541,Sold,2850000.0,2650000.0,2013,2/15/13,7/19/13,84.0,2819 East Lake Sammamish Pkwy NE,Sammamish,...,John L. Scott - KMS Renton,(425) 227-9200,False,NE Sammamish,Sewer Connected,True,True,"Cable TV, Deck, Dock, Fenced-Partially, Gas Av...",Lake Sammamish,MLS
4,474604,Sold,2500000.0,2490000.0,2013,4/18/13,7/19/13,10.0,3931 E Lake Sammamish Pkwy NE,Sammamish,...,Coldwell Banker Bain,(425) 454-0470,False,NESSWD,Sewer Connected,True,True,"Boat House, Deck, Dock, Gas Available, High Sp...",East Lake Sammamish,MLS


In [885]:
# importing the testing data
path = '/Users/mummadi/Downloads/Sammamish-Test.csv'
test = pd.read_csv(path)
test.head()

,ML Number,Status,List Price,Sold Price,Year,List Date,Sold Date,DOM,Address,City Name,...,Selling Office Name,Selling Office Phone,Senior Exemption,Sewer Company,Sewer Type,Show Addressto Public,Show Map Link,Site Features,Community,Sale Type
0,1119129,Sold,600000,637500,2017,5/5/17,6/9/17,5,1346 222 Place NE,Sammamish,...,Coldwell Banker Bain,(425) 771-6444,False,NE Sammamish Sewer,Sewer Connected,True,True,"Deck, Gas Available, High Speed Internet",Sammamish,MLS
1,1113595,Sold,578000,627000,2017,4/27/17,6/8/17,12,540 227th Ct NE,Sammamish,...,The Cascade Team,(425) 395-4399,False,NaN,NaN,True,True,NaN,Sammamish,MLS
2,1075202,Sold,599950,625000,2017,3/1/17,3/31/17,5,816 210th Ave NE,Sammamish,...,HomeSmart RE Associates,(206) 523-7653,False,NaN,Septic,True,True,"Cable TV, Deck, Gas Available, High Speed Inte...",Inglewood,MLS
3,1079015,Sold,575000,625000,2017,2/22/17,3/28/17,6,22640 NE 4th Place #59,Sammamish,...,Windermere Real Estate/East,(425) 392-6600,False,NaN,NaN,True,True,NaN,Sammamish,MLS
4,1089639,Sold,575000,625000,2017,3/15/17,4/28/17,4,310 227th Lane NE #149,Sammamish,...,ALLPRO Real Estate,(206) 306-2511,False,NaN,NaN,True,True,NaN,Sammamish,MLS


In [886]:
# appended both test and train data for cleaning
home = train.append(test)

In [887]:
# shape of the dataset
home.shape

(5031, 109)

In [888]:
# took the subset of columns  which are important
home = home[['List Price','Sold Price','Zip Code','Year','Bedrooms','DOM','Bathrooms','Sq Foot','Age','Lot Size','Style','View','Garage','Pool','Heating and Cooling','Interior','Exterior','Other Features','Latitude','Longitude','Property Type','Building Condition','Bus Line Nearby','Energy Source','Fireplaces Total','Floor Covering','Foundation','Lot Details','Topography','Sewer Type','Site Features','Community']]

In [889]:
#shape of the dataset
home.shape

(5031, 32)

In [898]:
# summary of the dataset
home.describe()

,List Price,Sold Price,Zip Code,Year,Bedrooms,DOM,Bathrooms,Sq Foot,Lot Size,Latitude,Longitude,Fireplaces Total
count,5.028000e+03,5.028000e+03,5028.000000,5028.000000,5028.000000,4709.000000,5028.000000,5028.000000,5.007000e+03,5028.000000,5028.000000,4942.000000
mean,7.657069e+05,7.689052e+05,98055.899165,2014.769690,3.759944,31.652155,2.754077,2850.489459,2.993052e+04,47.605021,-122.031406,1.444759
std,3.651222e+05,3.546881e+05,1255.169824,1.305624,0.940994,51.760461,0.776916,1068.937960,1.013958e+05,0.021176,0.028402,0.729911
min,6.500000e+04,7.350000e+04,9075.000000,2013.000000,0.000000,1.000000,0.000000,0.000000,0.000000e+00,47.554073,-122.089547,0.000000
25%,5.500000e+05,5.580000e+05,98074.000000,2014.000000,3.000000,5.000000,2.500000,2089.000000,5.879000e+03,47.589300,-122.054508,1.000000
50%,7.250000e+05,7.300000e+05,98074.000000,2015.000000,4.000000,10.000000,2.500000,2856.000000,8.993000e+03,47.603051,-122.033684,1.000000
75%,8.990000e+05,9.092475e+05,98075.000000,2016.000000,4.000000,36.000000,3.250000,3550.000000,1.426250e+04,47.622250,-122.015286,2.000000
max,4.550000e+06,4.810000e+06,98704.000000,2017.000000,9.000000,923.000000,7.000000,10380.000000,1.079815e+06,47.656933,-121.965102,7.000000


In [907]:
# structure of the dataset
home.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5015 entries, 0 to 99
Data columns (total 32 columns):
List Price             5015 non-null float64
Sold Price             5015 non-null float64
Zip Code               5015 non-null float64
Year                   5015 non-null int64
Bedrooms               5015 non-null float64
DOM                    4709 non-null float64
Bathrooms              5015 non-null float64
Sq Foot                5015 non-null int64
Age                    5015 non-null object
Lot Size               4994 non-null float64
Style                  5015 non-null object
View                   3022 non-null object
Garage                 5015 non-null object
Pool                   100 non-null object
Heating and Cooling    5015 non-null object
Interior               4331 non-null object
Exterior               5015 non-null object
Other Features         4898 non-null object
Latitude               5015 non-null float64
Longitude              5015 non-null float64
Property 

### Data Cleaning

In [891]:


# finding the columns which has null values
nullcols = home.columns[home.isnull().any()]
nullcols


Index(['List Price', 'Zip Code', 'Bedrooms', 'DOM', 'Bathrooms', 'Age',
       'Lot Size', 'Style', 'View', 'Garage', 'Pool', 'Heating and Cooling',
       'Interior', 'Exterior', 'Other Features', 'Latitude', 'Longitude',
       'Building Condition', 'Bus Line Nearby', 'Energy Source',
       'Fireplaces Total', 'Floor Covering', 'Foundation', 'Lot Details',
       'Topography', 'Sewer Type', 'Site Features'],
      dtype='object')

In [906]:
# checking for Duplicates
home.drop_duplicates(inplace=True)
home.shape 


(5015, 32)

In [908]:
# imputing Null values in 
home["View"] = home["View"].replace(np.nan, 'No View', regex=True)
home["Pool"] = home["Pool"].replace(np.nan, "No Pool", regex = True)
home["Other Features"] = home["Other Features"].replace(np.nan,"Double Oven,Dryer,Garbage Disposal,Microwave,Range/Oven,Refrigerator,Trash Compactor,Washer,Dishwasher",regex = True)
home["Sewer Type"] = home["Sewer Type"].replace(np.nan, "No Type", regex = True)
home["Site Features"] = home["Site Features"].replace(np.nan, "No features", regex = True)
home["Bus Line Nearby"] = home["Bus Line Nearby"].replace(np.nan,"No Bus", regex = True)
home["Fireplaces Total"] = home["Fireplaces Total"].replace(np.nan,home["Fireplaces Total"].mean())
home["Topography"] = home["Topography"].replace(np.nan,"No topography",regex = True)
home["Lot Details"] = home["Lot Details"].replace(np.nan,"No details",regex = True)
home["Foundation"] = home["Foundation"].replace(np.nan,"No Foundation",regex = True)
home["Floor Covering"] = home["Floor Covering"].replace(np.nan,"floor",regex = True)
home["Interior"] = home["Interior"].replace(np.nan,"missing",regex = True)
home["DOM"] = home["DOM"].replace(np.nan,home["DOM"].mean())
home["Building Condition"] = home["Building Condition"].replace(np.nan,"no info",regex = True)
home["Lot Size"] = home["Lot Size"].replace(np.nan,home["Lot Size"].mean())


In [ ]:
# wrote a function to find the unique values in coulms 
def split_column(x):
    l = x.dropna(axis = 0)
    l = list(l)
    s = set(l)
#    len(s)
#    type(l)
    ln = len(s)
    i = 0
    new = []
    p = []
    for each in l : 
        new = each.split(",")
        for i in new:
            p.append(i)
    return(set(p))

In [923]:
# Columns created with unique features of site features
home['Arena-Indoor'] = pd.np.where(home["Site Features"].str.contains("Arena-Indoo"),"1","0")
home["Arena-Outdoor"] = pd.np.where(home["Site Features"].str.contains("Arena-Outdoor"),"1","0")
home["Athletic Court"] = pd.np.where(home["Site Features"].str.contains("Athletic Court"),"1","0")
home["Barn"] = pd.np.where(home["Site Features"].str.contains("Barn"),"1","0")
home["Boat House"] = pd.np.where(home["Site Features"].str.contains("Boat House"),"1","0")
home["Cabana/Gazebo"] = pd.np.where(home["Site Features"].str.contains("Cabana/Gazebo"),"1","0")
home["Cable TV"] = pd.np.where(home["Site Features"].str.contains("Cable TV"),"1","0")
home["Deck"] = pd.np.where(home["Site Features"].str.contains("Deck"),"1","0")
home["Disabled Access"] = pd.np.where(home["Site Features"].str.contains("Disabled Access"),"1","0")
home["Dock"] = pd.np.where(home["Site Features"].str.contains("Dock"),"1","0")
home["Dog Run"] = pd.np.where(home["Site Features"].str.contains("Dog Run"),"1","0")
home["Fenced-Fully"] = pd.np.where(home["Site Features"].str.contains("Fenced-Fully"),"1","0")
home["Fenced-Partially"] = pd.np.where(home["Site Features"].str.contains("Fenced-Partially"),"1","0")
home["Gas Available"] = pd.np.where(home["Site Features"].str.contains("Gas Available"),"1","0")
home["Gated Entry"] = pd.np.where(home["Site Features"].str.contains("Gated Entry"),"1","0")
home["Green House"] = pd.np.where(home["Site Features"].str.contains("Green House"),"1","0")
home["High Speed Internet"] = pd.np.where(home["Site Features"].str.contains("High Speed Internet"),"1","0")
home["Hot Tub/Spa"] = pd.np.where(home["Site Features"].str.contains("Hot Tub/Spa"),"1","0")
home["No features"] = pd.np.where(home["Site Features"].str.contains("No features"),"1","0")
home["Outbuildings"] = pd.np.where(home["Site Features"].str.contains("Outbuildings"),"1","0")
home["Patio"] = pd.np.where(home["Site Features"].str.contains("Patio"),"1","0")
home["Propane"] = pd.np.where(home["Site Features"].str.contains("Propane"),"1","0")
home["RV Parking"] = pd.np.where(home["Site Features"].str.contains("RV Parking"),"1","0")
home["Shop"] = pd.np.where(home["Site Features"].str.contains("Shop"),"1","0")
home["Sprinkler System"] = pd.np.where(home["Site Features"].str.contains("Sprinkler System"),"1","0")
home["Stable"] = pd.np.where(home["Site Features"].str.contains("Stable"),"1","0")

In [924]:
#  Columns created with unique features of Interior
home["2nd Kitchen"] = pd.np.where(home["Interior"].str.contains("2nd Kitchen"),"1","0")
home["2nd Master BR"] = pd.np.where(home["Interior"].str.contains("2nd Master BR"),"1","0")
home["Bath Off Master"] = pd.np.where(home["Interior"].str.contains("Bath Off Master"),"1","0")
home["Built-In Vacuum"] = pd.np.where(home["Interior"].str.contains("Built-In Vacuum"),"1","0")
home["Ceiling Fan(s)"] = pd.np.where(home["Interior"].str.contains("Ceiling Fan(s)"),"1","0")
home["Disabled Access"] = pd.np.where(home["Interior"].str.contains("Disabled Access"),"1","0")
home["Dbl Pane/Storm Windw"] = pd.np.where(home["Interior"].str.contains("Dbl Pane/Storm Windw"),"1","0")
home["Dining Room"] = pd.np.where(home["Interior"].str.contains("Dining Room"),"1","0")
home["Fireplace in Mstr BR"] = pd.np.where(home["Interior"].str.contains("Fireplace in Mstr BR"),"1","0")
home["French Doors"] = pd.np.where(home["Interior"].str.contains("French Doors"),"1","0")
home["High Tech Cabling"] = pd.np.where(home["Interior"].str.contains("High Tech Cabling"),"1","0")
home["Hot Tub/Spa"] = pd.np.where(home["Interior"].str.contains("Hot Tub/Spa"),"1","0")
home["Jetted Tub"] = pd.np.where(home["Interior"].str.contains("Jetted Tub"),"1","0")
home["Loft"] = pd.np.where(home["Interior"].str.contains("Loft"),"1","0")
home["Security System"] = pd.np.where(home["Interior"].str.contains("Security System"),"1","0")
home["Sauna"] = pd.np.where(home["Interior"].str.contains("Sauna"),"1","0")
home["Solarium/Atrium"] = pd.np.where(home["Interior"].str.contains("Solarium/Atrium"),"1","0")
home["Wet Bar"] = pd.np.where(home["Interior"].str.contains("Wet Bar"),"1","0")
home["Skylights"] = pd.np.where(home["Interior"].str.contains("Skylights"),"1","0")
home["Vaulted Ceilings"] = pd.np.where(home["Interior"].str.contains("Vaulted Ceilings"),"1","0")
home["Walk In Pantry"] = pd.np.where(home["Interior"].str.contains("Walk In Pantry"),"1","0")
home["Walk-in Closet"] = pd.np.where(home["Interior"].str.contains("Walk-in Closet"),"1","0")
home["Wine Cellar"] = pd.np.where(home["Interior"].str.contains("Wine Cellar"),"1","0")
home["Wired for Generator"] = pd.np.where(home["Interior"].str.contains("Wired for Generator"),"1","0")
home["missing"] = pd.np.where(home["Interior"].str.contains("missing"),"1","0")

In [925]:
# Columns created with unique features of Exterior
home["Brick"] = pd.np.where(home["Exterior"].str.contains("Brick"),"1","0")
home["Cement Planked"] = pd.np.where(home["Exterior"].str.contains("Cement Planked"),"1","0")
home["Cement/Concrete"] = pd.np.where(home["Exterior"].str.contains("Cement/Concrete"),"1","0")
home["Log"] = pd.np.where(home["Exterior"].str.contains("Log"),"1","0")
home["Metal/Vinyl"] = pd.np.where(home["Exterior"].str.contains("Metal/Vinyl"),"1","0")
home["See Remarks"] = pd.np.where(home["Exterior"].str.contains("See Remarks"),"1","0")
home["Stone"] = pd.np.where(home["Exterior"].str.contains("Stone"),"1","0")
home["Stucco"] = pd.np.where(home["Exterior"].str.contains("Stucco"),"1","0")
home["Wood"] = pd.np.where(home["Exterior"].str.contains("Wood"),"1","0")
home["Wood Products"] = pd.np.where(home["Exterior"].str.contains("Wood Products"),"1","0")

In [926]:
#  Columns created with unique features of Other Features
home["Dishwasher"] = pd.np.where(home["Other Features"].str.contains("Dishwasher"),"1","0")
home["Double Oven"] = pd.np.where(home["Other Features"].str.contains("Double Oven"),"1","0")
home["Dryer"] = pd.np.where(home["Other Features"].str.contains("Dryer"),"1","0")
home["Garbage Disposal"] = pd.np.where(home["Other Features"].str.contains("Garbage Disposal"),"1","0")
home["Microwave"] = pd.np.where(home["Other Features"].str.contains("Microwave"),"1","0")
home["Range/Oven"] = pd.np.where(home["Other Features"].str.contains("Range/Oven"),"1","0")
home["Refrigerator"] = pd.np.where(home["Other Features"].str.contains("Refrigerator"),"1","0")
home["See Remarks"] = pd.np.where(home["Other Features"].str.contains("See Remarks"),"1","0")
home["Trash Compactor"] = pd.np.where(home["Other Features"].str.contains("Trash Compactor"),"1","0")
home["Washer"] = pd.np.where(home["Other Features"].str.contains("Washer"),"1","0")

In [927]:
#  Columns created with unique Valuees of Tography
home["Pasture"] = pd.np.where(home["Topography"].str.contains("Pasture"),"1","0")
home["Brush"] = pd.np.where(home["Topography"].str.contains("Brush"),"1","0")
home["Equestrian"] = pd.np.where(home["Topography"].str.contains("Equestrian"),"1","0")
home["Fruit Trees"] = pd.np.where(home["Topography"].str.contains("Fruit Trees"),"1","0")
home["Garden Space"] = pd.np.where(home["Topography"].str.contains("Garden Space"),"1","0")
home["Level"] = pd.np.where(home["Topography"].str.contains("Level"),"1","0")
home["No topography"] = pd.np.where(home["Topography"].str.contains("No topography"),"1","0")
home["Partial Slope"] = pd.np.where(home["Topography"].str.contains("Partial Slope"),"1","0")
home["Rolling"] = pd.np.where(home["Topography"].str.contains("Rolling"),"1","0")
home["Sloped"] = pd.np.where(home["Topography"].str.contains("Sloped"),"1","0")
home["Steep Slope"] = pd.np.where(home["Topography"].str.contains("Steep Slope"),"1","0")
home["Terraces"] = pd.np.where(home["Topography"].str.contains("Terraces"),"1","0")
home["Wooded"] = pd.np.where(home["Topography"].str.contains("Wooded"),"1","0")

In [928]:
#  Columns created with unique Valuees of Floor Covering
home["Bamboo/Cork"] = pd.np.where(home["Floor Covering"].str.contains("Bamboo/Cork"),"1","0")
home["Ceramic Tile"] = pd.np.where(home["Floor Covering"].str.contains("Ceramic Tile"),"1","0")
home["Concrete"] = pd.np.where(home["Floor Covering"].str.contains("Concrete"),"1","0")
home["Fir/Softwood"] = pd.np.where(home["Floor Covering"].str.contains("Fir/Softwood"),"1","0")
home["Hardwood"] = pd.np.where(home["Floor Covering"].str.contains("Hardwood"),"1","0")
home["Laminate"] = pd.np.where(home["Floor Covering"].str.contains("Laminate"),"1","0")
home["Other Renewable"] = pd.np.where(home["Floor Covering"].str.contains("Other Renewable"),"1","0")
home["See Remarks"] = pd.np.where(home["Floor Covering"].str.contains("See Remarks"),"1","0")
home["Slate"] = pd.np.where(home["Floor Covering"].str.contains("Slate"),"1","0")
home["Vinyl"] = pd.np.where(home["Floor Covering"].str.contains("Vinyl"),"1","0")
home["Wall to Wall Carpet"] = pd.np.where(home["Floor Covering"].str.contains("Wall to Wall Carpet"),"1","0")
home["floor"] = pd.np.where(home["Floor Covering"].str.contains("floor"),"1","0")

In [929]:
##  Columns created with unique Values of Lot Details
home["Alley"] = pd.np.where(home["Lot Details"].str.contains("Alley"),"1","0")
home["Corner Lot"] = pd.np.where(home["Lot Details"].str.contains("Corner Lot"),"1","0")
home["Cul-de-sac"] = pd.np.where(home["Lot Details"].str.contains("Cul-de-sac"),"1","0")
home["Curbs"] = pd.np.where(home["Lot Details"].str.contains("Curbs"),"1","0")
home["Dead End Street"] = pd.np.where(home["Lot Details"].str.contains("Dead End Street"),"1","0")
home["Drought Res Landscpe"] = pd.np.where(home["Lot Details"].str.contains("Drought Res Landscpe"),"1","0")
home["High Voltage Line"] = pd.np.where(home["Lot Details"].str.contains("High Voltage Line"),"1","0")
home["No details"] = pd.np.where(home["Lot Details"].str.contains("No details"),"1","0")
home["Open Space"] = pd.np.where(home["Lot Details"].str.contains("Open Space"),"1","0")
home["Paved Street"] = pd.np.where(home["Lot Details"].str.contains("Paved Street"),"1","0")
home["Secluded"] = pd.np.where(home["Lot Details"].str.contains("Secluded"),"1","0")
home["Sidewalk"] = pd.np.where(home["Lot Details"].str.contains("Sidewalk"),"1","0")
home["Value In Land"] = pd.np.where(home["Lot Details"].str.contains("Value In Land"),"1","0")

In [930]:
# #  Columns created with unique Valuees of Sewer Type
home["Available"] = pd.np.where(home["Sewer Type"].str.contains("Available"),"1","0")
home["No Type"] = pd.np.where(home["Sewer Type"].str.contains("No Type"),"1","0")
home["None"] = pd.np.where(home["Sewer Type"].str.contains("None"),"1","0")
home["Septic"] = pd.np.where(home["Sewer Type"].str.contains("Septic"),"1","0")
home["Sewer Connected"] = pd.np.where(home["Sewer Type"].str.contains("Sewer Connected"),"1","0")


In [931]:
#  Columns created with unique Valuees of ENergy source
home["Solar (Unspecified)"] = pd.np.where(home["Energy Source"].str.contains("Solar (Unspecified)"),"1","0")
home["Electric"] = pd.np.where(home["Energy Source"].str.contains("Electric"),"1","0")
home["Geothermal"] = pd.np.where(home["Energy Source"].str.contains("Geothermal"),"1","0")
home["Ground Source"] = pd.np.where(home["Energy Source"].str.contains("Ground Source"),"1","0")
home["Natural Gas"] = pd.np.where(home["Energy Source"].str.contains("Natural Gas"),"1","0")
home["Oil"] = pd.np.where(home["Energy Source"].str.contains("Oil"),"1","0")
home["Pellet"] = pd.np.where(home["Energy Source"].str.contains("Pellet"),"1","0")
home["Propane"] = pd.np.where(home["Energy Source"].str.contains("Propane"),"1","0")
home["See Remarks"] = pd.np.where(home["Energy Source"].str.contains("See Remarks"),"1","0")
home["Solar PV"] = pd.np.where(home["Energy Source"].str.contains("Solar PV"),"1","0")
home["Wood"] = pd.np.where(home["Energy Source"].str.contains("Wood"),"1","0")



In [932]:
##  Columns created with unique Valuees of Foundation
home["Concrete Block"] = pd.np.where(home["Foundation"].str.contains("Concrete Block"),"1","0")
home["Concrete Ribbon"] = pd.np.where(home["Foundation"].str.contains("Concrete Ribbon"),"1","0")
home["Concrete Slab"] = pd.np.where(home["Foundation"].str.contains("Concrete Slab"),"1","0")
home["No Foundation"] = pd.np.where(home["Foundation"].str.contains("No Foundation"),"1","0")
home["Post & Block"] = pd.np.where(home["Foundation"].str.contains("Post & Block"),"1","0")
home["Post & Pillar"] = pd.np.where(home["Foundation"].str.contains("Post & Pillar"),"1","0")
home["Poured Concrete"] = pd.np.where(home["Foundation"].str.contains("Poured Concrete"),"1","0")
home["See Remark"] = pd.np.where(home["Foundation"].str.contains("See Remark"),"1","0")
home["Slab"] = pd.np.where(home["Foundation"].str.contains("Slab"),"1","0")
home["Tie Down"] = pd.np.where(home["Foundation"].str.contains("Tie Down"),"1","0")



In [933]:
# #  Columns created with unique Valuees of Building Condition
home["Average"] = pd.np.where(home["Building Condition"].str.contains("Average"),"1","0")
home["Fair"] = pd.np.where(home["Building Condition"].str.contains("Fair"),"1","0")
home["Fixer"] = pd.np.where(home["Building Condition"].str.contains("Fixer"),"1","0")
home["Good"] = pd.np.where(home["Building Condition"].str.contains("Good"),"1","0")
home["Remodeled"] = pd.np.where(home["Building Condition"].str.contains("Remodeled"),"1","0")
home["Restored"] = pd.np.where(home["Building Condition"].str.contains("Restored"),"1","0")
home["Under Construction"] = pd.np.where(home["Building Condition"].str.contains("Under Construction"),"1","0")
home["Very Good"] = pd.np.where(home["Building Condition"].str.contains("Very Good"),"1","0")
home["no info"] = pd.np.where(home["Building Condition"].str.contains("no info"),"1","0")


In [934]:
# #  Columns created with unique Valuees of Heating and Cooling
home["Baseboard"] = pd.np.where(home["Heating and Cooling"].str.contains("Baseboard"),"1","0")
home["Central A/C"] = pd.np.where(home["Heating and Cooling"].str.contains("Central A/C"),"1","0")
home["Ductless HP-Mini Split"] = pd.np.where(home["Heating and Cooling"].str.contains("Ductless HP-Mini Split"),"1","0")
home["Forced Air"] = pd.np.where(home["Heating and Cooling"].str.contains("Forced Air"),"1","0")
home["Heat Pump"] = pd.np.where(home["Heating and Cooling"].str.contains("Heat Pump"),"1","0")
home["High Efficiency (Unspecified)"] = pd.np.where(home["Heating and Cooling"].str.contains("High Efficiency (Unspecified)"),"1","0")
home["Hot Water Recirc Pump"] = pd.np.where(home["Heating and Cooling"].str.contains("Hot Water Recirc Pump"),"1","0")
home["Insert"] = pd.np.where(home["Heating and Cooling"].str.contains("Insert"),"1","0")
home["None"] = pd.np.where(home["Heating and Cooling"].str.contains("None"),"1","0")
home["Other-See Remarks"] = pd.np.where(home["Heating and Cooling"].str.contains("Other-See Remarks"),"1","0")
home["Radiant"] = pd.np.where(home["Heating and Cooling"].str.contains("Radiant"),"1","0")
home["Radiator"] = pd.np.where(home["Heating and Cooling"].str.contains("Radiator"),"1","0")
home["Stove/Free Standing"] = pd.np.where(home["Heating and Cooling"].str.contains("Stove/Free Standing"),"1","0")
home["Tankless Water Heater"] = pd.np.where(home["Heating and Cooling"].str.contains("ATankless Water Heater"),"1","0")
home["Wall"] = pd.np.where(home["Heating and Cooling"].str.contains("Wall"),"1","0")
home["Window Unit A/C"] = pd.np.where(home["Heating and Cooling"].str.contains("Window Unit A/C"),"1","0")
home["HRV/ERV System"] = pd.np.where(home["Heating and Cooling"].str.contains("HRV/ERV System"),"1","0")
home["HEPA Air Filtration"] = pd.np.where(home["Heating and Cooling"].str.contains("HEPA Air Filtration"),"1","0")


In [935]:
# Columns created with unique Valuees of Pool
home["Above Ground"] = pd.np.where(home["Pool"].str.contains("Above Ground"),"1","0")
home["Community"] = pd.np.where(home["Pool"].str.contains("Community"),"1","0")
home["In-Ground"] = pd.np.where(home["Pool"].str.contains("In-Ground"),"1","0")
home["Indoor"] = pd.np.where(home["Pool"].str.contains("Indoor"),"1","0")
home["No Pool"] = pd.np.where(home["Pool"].str.contains("No Pool"),"1","0")

In [936]:
# Columns created with unique Valuees of garage
home["Carport"] = pd.np.where(home["Garage"].str.contains("Carport"),"1","0")
home["Carport-Attached"] = pd.np.where(home["Garage"].str.contains("Carport-Attached"),"1","0")
home["Common Garage"] = pd.np.where(home["Garage"].str.contains("Common Garage"),"1","0")
home["Garage-Attached"] = pd.np.where(home["Garage"].str.contains("Garage-Attached"),"1","0")
home["Garage-Detached"] = pd.np.where(home["Garage"].str.contains("Garage-Detached"),"1","0")
home["Individual Garage"] = pd.np.where(home["Garage"].str.contains("Individual Garage"),"1","0")
home["None"] = pd.np.where(home["Garage"].str.contains("None"),"1","0")
home["Off Street"] = pd.np.where(home["Garage"].str.contains("Off Street"),"1","0")
home["Uncovered"] = pd.np.where(home["Garage"].str.contains("Uncovered"),"1","0")

In [937]:
# Columns created with unique Valuees of View
home["Lake"] = pd.np.where(home["View"].str.contains("Lake"),"1","0")
home["Mountain"] = pd.np.where(home["View"].str.contains("Mountain"),"1","0")
home["Partial"] = pd.np.where(home["View"].str.contains("Partial"),"1","0")
home["See Remarks"] = pd.np.where(home["View"].str.contains("See Remarks"),"1","0")
home["Territorial"] = pd.np.where(home["View"].str.contains("Territorial"),"1","0")
home["No View"] = pd.np.where(home["View"].str.contains("No View"),"1","0")
home["Bay"] = pd.np.where(home["View"].str.contains("Bay"),"1","0")
home["Canal"] = pd.np.where(home["View"].str.contains("Canal"),"1","0")
home["City"] = pd.np.where(home["View"].str.contains("City"),"1","0")
home["Golf Course"] = pd.np.where(home["View"].str.contains("Golf Course"),"1","0")


In [938]:
#  Columns created with unique Valuees of Style
home["10 - 1 Story"] = pd.np.where(home["Style"].str.contains("10 - 1 Story"),"1","0")
home["11 - 1 1/2 Story"] = pd.np.where(home["Style"].str.contains("11 - 1 1/2 Story"),"1","0")
home["12 - 2 Story"] = pd.np.where(home["Style"].str.contains("12 - 2 Story"),"1","0")
home["13 - Tri-Level"] = pd.np.where(home["Style"].str.contains("13 - Tri-Level"),"1","0")
home["14 - Split Entry"] = pd.np.where(home["Style"].str.contains("14 - Split Entry"),"1","0")
home["15 - Multi Level"] = pd.np.where(home["Style"].str.contains("15 - Multi Level"),"1","0")
home["16 - 1 Story w/Bsmnt."] = pd.np.where(home["Style"].str.contains("16 - 1 Story w/Bsmnt."),"1","0")
home["17 - 1 1/2 Stry w/Bsmt"] = pd.np.where(home["Style"].str.contains("17 - 1 1/2 Stry w/Bsmt"),"1","0")
home["18 - 2 Stories w/Bsmnt"] = pd.np.where(home["Style"].str.contains("18 - 2 Stories w/Bsmnt"),"1","0")
home["20 - Manuf-Single Wide"] = pd.np.where(home["Style"].str.contains("20 - Manuf-Single Wide"),"1","0")
home["21 - Manuf-Double Wide"] = pd.np.where(home["Style"].str.contains("21 - Manuf-Double Wide"),"1","0")
home["30 - Condo (1 Level)"] = pd.np.where(home["Style"].str.contains("30 - Condo (1 Level)"),"1","0")
home["31 - Condo (2 Levels)"] = pd.np.where(home["Style"].str.contains("31 - Condo (2 Levels)"),"1","0")
home["32 - Townhouse"] = pd.np.where(home["Style"].str.contains("32 - Townhouse"),"1","0")
home["34 - Condo (3 Levels)"] = pd.np.where(home["Style"].str.contains("34 - Condo (3 Levels)"),"1","0")

In [939]:
#  Dummie created for busline near by
bus_line = pd.get_dummies(home["Bus Line Nearby"]).apply(pd.to_numeric)

home = pd.concat([home,bus_line], axis=1)
home.head()

,List Price,Sold Price,Zip Code,Year,Bedrooms,DOM,Bathrooms,Sq Foot,Age,Lot Size,...,18 - 2 Stories w/Bsmnt,20 - Manuf-Single Wide,21 - Manuf-Double Wide,30 - Condo (1 Level),31 - Condo (2 Levels),32 - Townhouse,34 - Condo (3 Levels),False,True,No Bus
0,4499000.0,4036000.0,98074.0,2013,4.0,117.000000,5.75,7230,2000.0,978070.000000,...,0,0,0,0,0,0,0,0,1,0
1,3700000.0,3685000.0,98074.0,2013,4.0,31.652155,4.25,7110,2000.0,24449.000000,...,0,0,0,0,0,0,0,0,0,1
2,2895000.0,2772500.0,98075.0,2013,4.0,25.000000,4.25,4710,2000.0,17176.000000,...,0,0,0,0,0,0,0,0,0,1
3,2850000.0,2650000.0,98074.0,2013,4.0,84.000000,5.75,5560,2000.0,9312.000000,...,1,0,0,0,0,0,0,0,1,0
4,2500000.0,2490000.0,98074.0,2013,4.0,10.000000,4.00,4430,2000.0,14556.000000,...,0,0,0,0,0,0,0,0,0,1
5,2498000.0,2400000.0,98074.0,2013,5.0,44.000000,3.50,4382,2000.0,30008.430316,...,0,0,0,0,0,0,0,0,1,0
6,2475000.0,2320000.0,98075.0,2013,4.0,39.000000,4.25,5250,2000.0,103237.000000,...,0,0,0,0,0,0,0,0,1,0
7,2348000.0,2305025.0,98074.0,2013,4.0,53.000000,4.25,4306,2000.0,8780.000000,...,0,0,0,0,0,0,0,0,0,1
8,2265000.0,2160200.0,98074.0,2013,5.0,48.000000,5.00,6280,2000.0,83813.000000,...,0,0,0,0,0,0,0,0,1,0
9,2349000.0,2115000.0,98074.0,2013,3.0,29.000000,3.25,3260,2000.0,8854.000000,...,0,0,0,0,0,0,0,0,1,0


In [948]:
# Dropped the columns in order to eleminate the redundency
new = home

# applied logarithm for sold and list price
new["List Price"] = np.log(new["List Price"])
new["Sold Price"] = np.log(new["Sold Price"])


# Dropped the columns in order to eleminate the redundency

new = new.drop(new[["Site Features","Interior","Exterior","Other Features","Property Type","Bus Line Nearby","Building Condition","Heating and Cooling","Pool","Garage","View","Style","Topography","Floor Covering","Lot Details","Sewer Type","Energy Source","Foundation"]],axis =1)


In [ ]:
# Splitting the Data

train = new.iloc[:4915,:]
X_train = train.drop("Sold Price",axis =1)
y_train  = train["Sold Price"]
test = new.iloc[4915:,:]
X_test = test.drop("Sold Price",axis = 1)
y_test = test["Sold Price"]

In [949]:
# Printing the shape train and test datset
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
new.info()

(4915, 209)
(100, 209)
(4915,)
(100,)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5015 entries, 0 to 99
Columns: 210 entries, List Price to No Bus
dtypes: float64(10), int64(2), object(192), uint8(6)
memory usage: 7.9+ MB


In [965]:
#  Linear Regression

# Importing linear model 
from sklearn import datasets, linear_model
from sklearn import metrics
from math import sqrt
import math

# linear model
lm = linear_model.LinearRegression()
# fitting model
model = lm.fit(X_train, y_train)
# predicting with test data
predictions = lm.predict(X_test)

# R2 Value
print('Linear Regression R squared": %.4f' % lm.score(X_test, y_test))


Linear Regression R squared": 0.9577


In [955]:
# importing MAE, MSE error functions from sklearn

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# delogging by adding exxponential to the predicted values
#MAE 
MAE = mean_absolute_error(np.exp(y_test), np.exp(predictions))
print("MAE:",MAE)

#MSE
meanSquaredError= mean_squared_error(np.exp(y_test), np.exp(predictions))
print("MSE:", meanSquaredError)

#RMSE
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)

MAE: 20733.536123225513
MSE: 661246516.5209483
RMSE: 25714.714008150047


In [956]:
# Function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [958]:
# MAPE
mean_absolute_percentage_error(np.exp(y_test), np.exp(predictions))

4.5668069887634

In [962]:
# Coefficients  and intercept values with linear Regression
print('intercept:', model.intercept_)
print('coefficients:', model.coef_)

intercept: -18.216893660384716
coefficients: [ 9.56701895e-01 -9.66839713e-08  7.15645903e-03 -6.53124833e-05
 -2.05084067e-04  4.78883548e-03 -8.24026969e-07  6.21131088e-04
  7.35224454e-10 -4.54944217e-02 -4.30534656e-02  9.05601679e-04
  2.15303063e-04  5.90656506e-02 -1.93432230e-02  3.00892220e-03
  8.55350194e-03 -2.85795067e-02  3.69708008e-03  2.94422663e-04
  1.16718957e-03 -1.08218996e-02  1.41295803e-02 -5.78170586e-03
  1.72882326e-04 -1.74261154e-03  1.32142353e-03 -2.72955734e-03
 -3.37892003e-03 -2.04081187e-04 -2.55583167e-03  6.81187476e-03
  4.35019864e-03  1.51797126e-03  3.06182576e-03 -7.57470695e-04
 -5.12870658e-03  3.17040586e-03 -2.04094237e-02 -1.49172705e-02
  5.58033043e-04 -3.73681477e-03 -2.30799094e-03  2.74086309e-16
  1.63568528e-03  3.00944154e-03 -1.02487837e-03 -7.81100209e-04
 -3.21387054e-04  6.33103678e-04  2.06915119e-03 -5.47338000e-04
 -1.92502318e-02 -4.74342025e-03 -6.04727937e-03 -2.22845757e-03
  1.96523253e-03 -3.21993330e-03  6.53743111e

In [963]:
# Regression with Random Forest

from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_model = forest_reg.fit(X_train, y_train)
y_pred = forest_reg.predict(X_test)
print('Random Forest R squared": %.4f' % forest_reg.score(X_test, y_test))

Random Forest R squared": 0.9534


In [960]:
# delogging by adding exxponential to the predicted values
#MAE 
MAE = mean_absolute_error(np.exp(y_test), np.exp(y_pred))
print("MAE:",MAE)

#MSE
meanSquaredError= mean_squared_error(np.exp(y_test), np.exp(y_pred))
print("MSE:", meanSquaredError)

#RMSE
rootMeanSquaredError = sqrt(meanSquaredError)
print("RMSE:", rootMeanSquaredError)

# MAPE
MAPE = mean_absolute_percentage_error(np.exp(y_test), np.exp(predictions))
print("MAPE:", MAPE)

MAE: 21015.15167733272
MSE: 754721761.8104838
RMSE: 27472.199799260412
MAPE: 4.5668069887634
